Steps to build the Loan Eligibility model

1. Loading the dataset
2. Pre-processing the dataset
3. Building the Loan Prediction model

Loading the dataset

In [1]:
# importing libraries
import pandas as pd

# loading the dataset
train = pd.read_csv('loan_data.csv')
train.head()

,Gender,Married,ApplicantIncome,LoanAmount,Loan_Status
0,Male,Yes,4583,128000.0,N
1,Male,Yes,3000,66000.0,Y
2,Male,Yes,2583,120000.0,Y
3,Male,No,6000,141000.0,Y
4,Male,Yes,5417,267000.0,Y


2. Pre-processing the dataset

In [2]:
# converting categories into numbers
train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})

In [3]:
# separating dependent and independent variables
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount']]
y = train.Loan_Status

Building the Loan Prediction model

In [4]:
# importing machine learning model
from sklearn.linear_model import LogisticRegression

In [5]:
# training the logistic regression model
model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [6]:
# saving the model
import pickle
pickle_out = open("classifier.pkl", mode = "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

Deploying the machine learning model using streamlit

1. Building the Frontend of the application
2. Loading and Pre-processing the data
3. Building the Machine Learning model to automate Loan Eligibility
4. Deploying the application

Building the Frontend of the application

1.1 Install Required Libraries<br>
1.2 Creating the Frontend of the app using Streamlit

In [7]:
# installing pyngrok
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [8]:
# installing streamlit
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


1.2. Creating the frontend of the app using streamlit

In [20]:
# creating the script
%%writefile app.py

# importing required libraries
import pickle
import streamlit as st

# loading the trained model
pickle_in = open('classifier.pkl', 'rb')
classifier = pickle.load(pickle_in)

# this is the main function in which we define our app
def main():
    # header of the page
    html_temp = """
    <div style ="background-color:yellow;padding:13px">
    <h1 style ="color:black;text-align:center;">Check your Loan Eligibility</h1>
    </div>
    """
    st.markdown(html_temp, unsafe_allow_html = True)

    # following lines create boxes in which user can enter data required to make prediction
    Gender = st.selectbox('Gender',("Male","Female","Other"))
    Married = st.selectbox('Marital Status',("Unmarried","Married","Other"))
    ApplicantIncome = st.number_input("Monthly Income in Rupees")
    LoanAmount = st.number_input("Loan Amount in Rupees")
    result =""

    # when 'Check' is clicked, make the prediction and store it
    if st.button("Check"):
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount)
        st.success('Your loan is {}'.format(result))

# defining the function which will make the prediction using the data which the user inputs
def prediction(Gender, Married, ApplicantIncome, LoanAmount):

    # 2. Loading and Pre-processing the data

    if Gender == "Male":
        Gender = 0
    else:
        Gender = 1

    if Married == "Married":
        Married = 1
    else:
        Married = 0
        prediction = classifier.predict(
        [[Gender, Married, ApplicantIncome, LoanAmount]])

    if prediction == 0:
        pred = 'Rejected'
    else:
        pred = 'Approved'
    return pred

if __name__=='__main__':
    main()

Overwriting app.py


In [ ]:
4. Deploying the application

In [21]:
# running the app
!streamlit run app.py &>/dev/null&

In [22]:
!ngrok config add-authtoken 2VybCaUlfwvLM19kOeKFwvmuFzP_M99kXFM2Hmc5kFzVLWqP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [14]:
!kill -9 "$(pgrep ngrok)"

In [23]:
# making the locally-hosted web application to be publicly accessible
from pyngrok import ngrok

public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "https://2ce2-35-231-20-187.ngrok-free.app" -> "http://localhost:8501">